# Session 6 (Bonus): Entity (dis)appearance and simple eco-evolutionary simulation

In all precedent sessions, we saw how to control a simulation from a notebook. This was done either by directly controlling the agents, by modifying their behaviors, routines, manipulating their sensors or their diets. We also saw how to control the environment to make resources spawn, start the eating mechanism of agents etc.

Now we are going to see how we can combine all these features to implement a simple Eco-Evolutionary simulation ! 

An eco-evolutionary simulation is a virtual environment where agents interact with each other and their surroundings. Over time, these agents adapt and evolve based on environmental challenges and competition for resources. The simulation implements both ecological processes (like food gathering or predator-prey relationships) and evolutionary changes (agent reproduction and death). To make things a bit simpler, however, we will not really consider the evolutionary aspect here, only the reproductive one.

On the practical side, you will learn how to:

- Making entities dynamically appear or disappear in the environment
- Attach routines to any entity type (in previous sessions we only attached routines to agents, here we will also attach them to objects)
- Attach routines to the controller (this can be useful to define environmental routines that are not attached to a specific entity, e.g. to implement a reproduction mechanism)

We will start by importing the necessary modules and functions as usual. In this session, we will also use the `numpy` library, which is a powerful library for numerical computing in Python. It provides support for arrays and matrices, along with a collection of mathematical functions to operate on these arrays. You can check the documentation [here](https://numpy.org/doc/stable/). 

We will also use the `matplotlib` library to change the colors of the entities thanks to the `colors` module. You can check the documentation [here](https://matplotlib.org/stable/contents.html). 

In [1]:
import numpy as np
import matplotlib.colors as colors

from vivarium.utils.handle_server_interface import start_server_and_interface, stop_server_and_interface
from vivarium.controllers.notebook_controller import NotebookController

In [ ]:
start_server_and_interface(scene_name="session_6")

Create and run the controller:

In [ ]:
controller = NotebookController()
controller.run()

Then, print the different subtypes present in the simulation, you will need this information later:

In [ ]:
controller.print_subtypes_list()

As in session 4, we will work with `agents`, `resources`, as well as small and big `obstacles`. We will also define classical braitenberg behaviors in advance, in order to react to other agents' presence, as well as the `obstacle_avoidance` and `foraging` behaviors.

In [5]:
def fear(agent):
    left, right = agent.sensors(sensed_entities=["agents"])
    left_wheel = left
    right_wheel = right
    return left_wheel, right_wheel

def aggression(agent):
    left, right = agent.sensors(sensed_entities=["agents"])
    left_wheel = right
    right_wheel = left
    return left_wheel, right_wheel

def love(agent):
    left, right = agent.sensors(sensed_entities=["agents"])
    left_wheel = 1 - left
    right_wheel = 1 - right   
    return left_wheel, right_wheel

def shyness(agent):
    left, right = agent.sensors(sensed_entities=["agents"])
    left_wheel = 1 - right
    right_wheel = 1 - left   
    return left_wheel, right_wheel

def obstacle_avoidance(agent):
    left, right = agent.sensors(sensed_entities=["s_obstacles", "b_obstacles"])
    left_wheel = 1 - right
    right_wheel = 1 - left   
    return left_wheel, right_wheel

def foraging(agent):
    left, right = agent.sensors(sensed_entities=["resources"])
    left_activation = right
    right_activation = left
    return left_activation, right_activation

Additionally, the simiulation might be slow with the default parameters as there are a lot of entities in this scene. You can speed up the simulation by increasing the `Num steps lax` parameter in the web interface (e.g from 6 to 12 or so, see session 4).

## Existing and non-existing entities

In this session, we will dive deeper into the concept of existing and non existing entities. We already briefly saw this in the end of session 3, where we added an agent to the simulation. Let's first start printing the total number of agents in the simulation:

In [ ]:
n_agents = len(controller.agents)
print(f"Total Number of agents (existing and non-existing): {n_agents}")

The cell above indicates that there are 10 agents in total in our simulation. But in the interface can see only 4 agents (if not, you can uncheck and check again the "Hide non existing" checkbox in the SIMULATOR tab of the web interface). If we only see 4 of them at the moment, it is because 6 of them are marked as "non existing", which make them invisible to us in the web interface, but also prevents them from interacting with any other entity in the scene (they cannot be sensed or collide with other entities). We can check this with the following command and see which agents are existing or non-existing by accessing their `exists` attribute:

In [ ]:
for agent in controller.agents:
    print(f"agent {agent.idx}: exists = {agent.exists}")

We can indeed observe that only the first 4 agents are existing, they correspond to those that are visible in the interface. We can modify the `existing` attribute by an an agent in order to make a non-existing agent exists, or vice-versa. For instance, we see above that agent 4 is non-existing. Let's make it exist with:

In [8]:
agent_idx = 4 # index of a non-existing agent
agent = controller.agents[agent_idx]
agent.exists = True # make the agent exist

Now you should see 5 agents in the interface.

You can try to now make it non-existing again, simply by replacing `True` with `False` in the last cell above and re-executing it. Then make it re-exist again.

We also provide a special function from the controller that handles this for us: `spawn_entity`. It can be used either on an agent or an object. You just need to provide the index of the entity you want to spawn, and can additionally pass a `position` argument in order to make the entity appear at a specific position. For instance if we want to spawn the agent 5 at the coordinates (150, 150), we can do it with the following command:

In [ ]:
agent_idx = 5 # index of a non-existing agent
new_agent = controller.spawn_entity(agent_idx, position=[150, 150])

The `controller.spawn` method returns the newly existing entity, which in the cell above we store in `new_agent`.

The same mechanisms can be used to spawn objects. Let's have a look at which objects are existing in our current simulation:

In [ ]:
for object in controller.objects:
    print(f"object {object.idx}: exists = {object.exists}")

We see that e.g. the object with index 17 does not exist. Let's make it appear at position (100, 50) with:

In [11]:
object_idx = 17 # index of a non-existing object
new_object = controller.spawn_entity(object_idx, position=[100, 50])

**Important note: Never position two entities at the exact same place or the simulation will crash.** The reason is that the physics engine will then generate infinite forces.

Less important and optional note: Notice that the indexes of objects above (`object.idx`) do not start at 0 as it was the case for the agents, but instead start at 10. The reason is that indexes emcompass all entites, i.e. both agents and objects. Agents occupy the first indexes : since we have 10 agents in total, their indexes are from 0 to 9. Objects occupy the next indexes (after the agents), i.e. starting at 10. Since we have 20 objects in total in this simulation (including existing and non-existing ones), their indexes are from 10 to 29.)

Similarly, there is a method in the controller to remove entites from the scene, called `remove_entity`. It also requires the index of the entity you want to remove. For instance, if we want to remove agent 5, we can do it with the following command:

In [12]:
agent_idx = 5 # idx of an alive agent
controller.remove_entity(agent_idx)

The methods we have just seen can be used for example to make agents be "born" and "die" in the environment. We will experience this later in this session.

If you need to iterate through only existing agents, you can can use:

In [ ]:
print("Getting existing agents with 'controller.existing_agents': ")
for agent in controller.existing_agents:
    print(f'Agent {agent.idx} is existing')

And similarly for non-existing ones:

In [ ]:
print("Getting non existing agents with 'controller.non_existing_agents': ")
for agent in controller.non_existing_agents:
    print(f'Agent {agent.idx} does not exist')

For example, these shortcuts can be used to attach behaviors only to existing agents. You can technically also attach them to non-existing agents but it won't have any effect, since their behaviors won't be executed (however it might slow down the simulation):

In [15]:
for agent in controller.existing_agents:
    agent.attach_behavior(obstacle_avoidance)
    agent.attach_behavior(love)

If we print the behaviors of all agents in the simulation, we can see that the first 4 agents indeed have the same behaviors attached to them, and the 6 other ones have no behavior attached to them.

In [ ]:
for agent in controller.agents:
    agent.print_behaviors()

Now let's stop the behaviors and the motors of the agents in order to proceed to the next part of the session. 

In [17]:
for agent in controller.agents:
    agent.detach_all_behaviors(stop_motors=True)

## Understanding Entities routines

We already saw the routine mechanism in the last session. It was used to e.g. update the energy level of the agents. In this session, we are going to see how to use routines on any kind of entities, i.e. also on objects. 

Let's first recall how routines are defined and attached (more detailed in session 4). They are pretty similar to behaviors. They are defined as functions with a single argument representing the entity they will be attached to (either an agent or an object). You can attach them with the `attach_routine` method of the entity (you can also precise its interval of execution with the `interval` argument), and detach them with the `detach_routine` method.

### Making an obstacle move

You can easily make an object follow a particular movement with the following routines (e.g move it to the left by 2 units every time the routine is called):

In [20]:
position_shift = 2.

# Define two routines:

# move the entity to the right by 2 units
def move_entity_right_routine(entity):
    entity.x_position += position_shift

# move the entity to the top by 2 units
def move_entity_top_routine(entity):
    entity.y_position += position_shift

Let's give an object specific characteristics:

In [21]:
obj = controller.objects[16]
obj.diameter = 10.
obj.color = "black"

You can then attach a first routine, with an interval of 1 for example (the routine will be executed at each simulation step):

In [22]:
obj.attach_routine(move_entity_right_routine, interval=1)

Now you should see this object move left to right.

You can combine this with a routine to go top which, combined with the previous routine, will make object move diagonally. 

In [23]:
obj.attach_routine(move_entity_top_routine, interval=1)

You can then detach the routines from the object : 

In [24]:
obj.detach_routine(move_entity_right_routine)
obj.detach_routine(move_entity_top_routine)

You can try implementing a similar mechanism with other kind of objects, like resources.

### Making an entity change color periodically :

Here we will present another example of a routine that you can use in your future projects. It will manipulate the color of entities. For instance, we will make an agent change its color every 2 simulation steps.

First, we need to better understand how we can control the colors of the entities in the simulation. You can actually set them as strings, but they are encoded in [hexedecimal format](https://htmlcolorcodes.com/) (HEX) within the entities, as we can see in the following cell:

In [ ]:
agent = controller.agents[0]
agent.color = "red"
agent_color = agent.color
print(f"Agent color is: {agent_color}")

Therefore, while you can specify entitiy colors using color names (e.g. `"red"` above), you will need to use the hexadecimal format for e.g. comparing colors. 

In the following cell, we show how to manipulate the colors with the `matplotlib.colors` module. You can use the `to_hex` function to convert a color (either a string or a [RGB value](https://www.rapidtables.com/web/color/RGB_Color.html)) to its hexadecimal representation.

In [ ]:
# blue color
print("Blue color")
print(f"Transforming blue text color to HEX: {colors.to_hex('blue')}")
print(f"Transforming blue RGB color to HEX: {colors.to_hex((0.0, 0.0, 1.0))}")

# red color
print("\nRed color")
print(f"Transforming red text color to RGB: {colors.to_rgb('red')}")
print(f"Transforming red HEX color to RGB: {colors.to_rgb('#ff0000')}")

Now that you know how to get the hex code of a color, you can make an entity change color periodically with the following routine :

In [27]:
def change_color(entity):
    # if the entity hex color is red, change it to blue
    if entity.color == colors.to_hex("red"):
        entity.color = "blue"
    # otherwise, change it to red
    else:
        entity.color = "red"

Then we attach it with an interval of 5 to the agent to make it change color every 5 simulation steps:

In [28]:
agent.attach_routine(change_color, interval=5)

In the interface, you will that the agent is now blinking between red and blue color. Let's stop it by detaching the routine:

In [29]:
agent.detach_routine(change_color)

This won't necessarly be useful for your next projects, but it's a good example of how you can manipulate entities in the simulation . You can also combine this with other routines to create more complex behaviors. For example, we could have this kind of routines attached to agents to indicate their health status (blinking when they have low energy), or to use selective sensing based on other agents' colors (see session 4).

Let's stop the controller to proceed to the next part of the session:

In [ ]:
controller.stop()

## Understanding Controller routines

We already saw how to implement and use routines for agents and objects, but we can also define routines that are not attached to specific entities. We call such routines *controller routines*. The routines of the controller are implemented and use in a very similar way as for entities. We define a controller routine as a Python function (similarly to entity routines we have seen before), but taking as argument the `controller` (instead of an agent or object as previously). The `controller.attach_routine` and `controller.detach_routine` methods manage its activation in the simulation. 

Let's use a controller routine to re-implement the eating mechanisms we used in previous sessions, enabling agents to "eat" resources when they are close to them. This is just for the sake of the exercice since we already saw a much more concise way to activate a pre-defined eating mechanism in session 3 (which behind the scene is actually implemented as a controller routine). 

An eating mechanism can be implemented in different ways, we will present one of them in the next cell. We start by defining the routine function, `example_eating_routine` here, with `controller` as an argument. Then, we iterate over the list of existing agents in order to make them "eat" resources next to them. At the level of one agent, we iterate over its list of eatable entities (`diet`). Then we check which entities corresponding to these types are existing and within the eating range of the agent, and we make them dissapear. Finally if an agent eats, we also reset its `time_since_feeding` to 0, and set its `ate` flag to True to signal he has eaten in the last simulation step.

You can take a look at the commented code below (don't worry, you don't need to understand everything in details, just the general idea).

In [31]:
def example_eating_routine(controller):
    # iterate over all existing agents to see if they can eat any entity
    for agent in controller.existing_agents:
        # iterate over all entity types in the agent diet
        for entity_type in agent.diet:
            # transform this eatable entity type label into an idx (e.g "resources" -> 1)
            entity_type = controller._subtype_label_to_idx[entity_type]
            # get the indexes of all entities in the simulation that correspond to this eatable entity type (excluding the agent itself)
            eatable_entities_idx = [ent.idx for ent in controller.all_entities if ent.subtype == entity_type and ent.idx != agent.idx]
            # get the distances between the agent and all eatable entities
            distances = agent.config.proximity_map_dist[eatable_entities_idx]
            # check if the distance between the agent and the eatable entities is less than the eating range of the agent
            in_range = distances < agent.eating_range
            
            # iterate over all eatable entities indexes to see if the agent can eat them
            for arr_idx, ent_idx in enumerate(eatable_entities_idx):
                # if the entity exists and is in range :
                if in_range[arr_idx] and controller.all_entities[ent_idx].exists:
                    # remove it from the simulation
                    controller.remove_entity(ent_idx)
                    # set the ate flag of the agent to True, and reset the time since feeding to 0
                    agent.time_since_feeding = 0
                    agent.ate = True

The routing function defined above rely on more advanced python concepts unseen in the last sessions, such as `list comprehension`, or the `enumerate` function. If you want to know more about it you can have a look at the [Python basics tutorial](./python_basics.ipynb).

First, run the controller again (we stopped it at the end of the last section): 

In [32]:
controller.run()

Let's attach the `example_eating_routine` we have defined above to the controller:

In [33]:
controller.attach_routine(example_eating_routine, interval=10)

You can also check the routines of the `controller` with the `print_routines` method. As for entities, the important thing is to check the `Active routines` of this print statement to see which routines are actually running in the controller.

In [ ]:
controller.print_routines()

Let's now define the diet of our agents (they will eat `resources`) and attach the `foraging` and `obstacle_avoidance` behaviors to them (we defined those behaviors at the start of this session):

In [35]:
for agent in controller.existing_agents:
    agent.diet = ["resources"]
    agent.attach_behavior(foraging)
    agent.attach_behavior(obstacle_avoidance)

You should now see the eating mechanism in action in the simulation: whenever an agent is close enough to a resource (the green squares), that resource will disappear.  

Note however that the `example_eating_routine` defined above might slow down the simulation a lot if there are many entities in the simulation. The reason is that it relies on two nested loops: a first one iterating over all existing agents, within which there is a second loop iterating over all entites. With many entities, this double iteration can be slow. If you don't need a custom eating mechanisms, we instead recommend to use the predefined one we introduced in session 3. To do this, first detach the `example_eating_routine`:

In [36]:
controller.detach_routine(example_eating_routine)

And start the built-in eating mechanism, which is much better optimized and run faster:

In [37]:
controller.start_eating_mechanism()

Also remember that you can start the automatic spawing of resources with:

In [38]:
controller.start_resources_apparition(interval=30)

Actually, these two mechanisms (eating and resource spawn) are also implemented as routines behind the scene. Let's stop them for now:

In [39]:
controller.stop_eating_mechanism()
controller.stop_resources_apparition()

And stop the behaviors and the motors of the agents.

In [40]:
for agent in controller.existing_agents:
    agent.detach_all_behaviors(stop_motors=True)

## Implementing reproduction with controller routines 

Now that you understood the basics of entities and controller routines, we are going to implement an agent reproduction mechanism in the simulation. Like small organisms, our agents will be able to produce offsprings next to them. 

To do so, we will need to manipulate existing and non existing agents. Let's do a quick recap on a few function and discover new features that will be useful for this part of the session.

If we want to make agents spawn, we first need to get a non existing agent in order to make it alive. We saw we can get the list of existing and non existing agents with the `existing_agents` and `non_existing_agents` attributes of the controller. We can actually get the indexes of the non existing agents this way:

In [ ]:
non_existing_agents_idx = [agent.idx for agent in controller.non_existing_agents]
print(non_existing_agents_idx)

Then, we want to pick a random index inside this list, using the `np.random.choice` function (note that if you execute the function several times, you will see that the index isn't the same everytime time):

In [ ]:
random_agent_idx = np.random.choice(non_existing_agents_idx)
print(random_agent_idx)

Now we can spawn this random non-existing agent. In order to implement a reproduction mechanism, we might want to spawn offsprings next to their parent. Let's say the agent 0 will reproduce, we can get its position with the following command :

In [ ]:
agent_idx = 0
# get agent position
position = [agent.x_position, agent.y_position]
print(f"agent position is {position}")

If we make an offspring spawn exactly at the same position, we will raise an error. This is because the physics engine will generate infinite forces if two entities are located at the exact same place. **(Important note : never place two entities at the exact same place or the simulation will crash.)**

So we need to spawn the new agent at a position that is slightly different from its parent position. To do so, we can add a small random value `epsilon` to the modify the position of the offspring with numpy. Here is a way to generate random values for the x and y axis between -5 and 5:

In [ ]:
# epsilon between -max_distance and +max_distance, for both x and y axes (i.e. 2 values)
max_epsilon_distance = 5
epsilon = np.random.uniform(-max_epsilon_distance, max_epsilon_distance, 2)
print(f"epsilon = {epsilon}")

In [ ]:
# create new position by adding epsilon to the current position
new_position = position + epsilon
print(f"original position: {position}")
print(f"new position: {new_position}")

## Implementing an asexual reproduction routine

With all the previous information, we can now implement a simple reproduction routine for agents. We will implement an asexual reproduction mechanism (i.e. no mating, like in e.g. bacteria), where an agent will spawn an offspring next to it when it eats a resource. 

To do so, we will use the `has_eaten` function of the agents, which returns to `True` when the agent has eaten since the last call of the function, `False` otherwise. We will also make the agent spawn an offspring with a smaller diameter than itself to make offsprings easily distinguishable. We will also attach behaviors to offsprings in order to make them move and eat.

In [46]:
# spawn the a new agent every time an agent has eaten

offspring_diameter = 7.

# Define a controller routine for asexual reproduction
def asexual_reproduction(controller):
    # iterate over all the existing agents in the controller
    for agent in controller.existing_agents: 
        # check if the agent has eaten
        if agent.has_eaten():
            # get the index of the non-existing agents to select one and make it spawn
            non_existing_agents_idx = [agent.idx for agent in controller.non_existing_agents]

            # check if there are non existing agents (if non_existing_agents_idx is an empty list, do nothing)
            if non_existing_agents_idx:
                # select one non-existing agent index randomly
                agent_idx = np.random.choice(non_existing_agents_idx)

                # add some noise to the position of the parent to create the offspring position
                epsilon = np.random.uniform(-max_epsilon_distance, max_epsilon_distance, 2)
                parent_position = [agent.x_position, agent.y_position]
                offspring_position = parent_position + epsilon

                # make the offspring spawn at the new position
                offspring = controller.spawn_entity(agent_idx, position=offspring_position) 

                # set the diameter of the offspring
                offspring.diameter = offspring_diameter

                # attach the behaviors to the offspring so it starts moving and foraging
                offspring.attach_behavior(obstacle_avoidance)
                offspring.attach_behavior(foraging)
                
                # Add resources to the diet of the offspring
                offspring.diet = ["resources"]

Let's start again the eating and resources spawning mechanisms, and attach the `obstacle_avoidance` and `foraging` behaviors to the agents. 

In [47]:
# update agents attributes and behaviors
for agent in controller.existing_agents:
    # set the ate flag to False to prevent the agent from spawning an offspring if they already ate before
    agent.ate = False
    agent.diet = ["resources"]
    agent.detach_all_behaviors()
    agent.attach_behavior(obstacle_avoidance)
    agent.attach_behavior(foraging)

# start the resource spawning and the eating mechanisms
controller.start_resources_apparition(interval=20)
controller.start_eating_mechanism()  # Note that this is the eating mechanism we used in session 3, which is better optimized that the one we defined above. 

Finally, let's attach the reproduction routine we just created:

In [48]:
controller.attach_routine(asexual_reproduction, interval=10)

Now you should have a simulation where agents procuce an offspring whenever they eat a resource! But the number of agents is still limited to 10, so it might be interesting to implement a mechanism to kill agents when they reach a certain age for example. If you want you can try to implement a dying mechanism by yourself using controller and entity routines.

Let's now detach all routines and behaviors:

In [ ]:
controller.detach_all_routines()

for agent in controller.agents:
    agent.detach_all_behaviors(stop_motors=True)

## Sexual reproduction with a controller routine

Nice, now we have agents that can spawn offsprings next to them when they eat. We could also think of other types of reproduction, for example if two agents are close to each other and they have both eaten, they can reproduce. This is loosely analogous to sexual reproduction in nature, where animals need to find a mate in order to reproduce. We can implement this with another controller routine.

First, let's remove some agents in the scene in order to be able to spawn new ones (because we currently have a maximum of 10 agents in the simulation). Do do this let's first remove all agents:

In [ ]:
for agent in controller.existing_agents:
    controller.remove_entity(agent.idx)

And reintroduce only the first 5 agents of the list:

In [51]:
for agent in controller.agents[0:5]:
    agent.exists = True

Let's define a new variable `reproduction_range` that specifies the maximum distance between two agents enabling them to reproduce:

In [52]:
reproduction_range = 30.

Now we can implement the reproduction routine. We will iterate over the list of existing agents, and for each agent, we will check if it can reproduce. If it is the case, we will check if it has any mate in its neighborhood to reproduce with. If it is the case, we will make them reproduce by spawning an offspring next to them:

In [53]:
def sexual_reproduction(controller):
    for agent in controller.existing_agents:
        if agent.has_eaten():
            # get the index of the other existing agents in the simulation
            other_existing_agents_idx = [ent.idx for ent in controller.existing_agents if ent.idx != agent.idx]
            # Compute the distances between the agent and all other existing agents
            distances = agent.config.proximity_map_dist[other_existing_agents_idx]
            # check if other agents are in the reproduction range
            in_reproduction_range = distances < reproduction_range
            # check if the agent has found a mate, if any other agent is in the reproduction range
            found_mate = np.any(in_reproduction_range)
        
            # need to have found a mate in order to reproduce
            if found_mate:
                dead_agents_idx = [agent.idx for agent in controller.non_existing_agents]
                if dead_agents_idx:
                    # same steps as the asexual reproduction routine to spawn the offspring
                    agent_idx = np.random.choice(dead_agents_idx)
                    epsilon = np.random.uniform(-max_epsilon_distance, max_epsilon_distance, 2)
                    position = [agent.x_position, agent.y_position]
                    new_position = position + epsilon

                    offspring = controller.spawn_entity(agent_idx, position=new_position) 
                    offspring.diameter = offspring_diameter
                    offspring.attach_behavior(obstacle_avoidance)
                    offspring.attach_behavior(foraging)

                    # set the can_reproduce flag to False for the parent and the offspring
                    #offspring.can_reproduce = False
                    #agent.can_reproduce = False
                    

You could easily improve this mechanism. For example, you could add other constraints for the reproduction, such as having agents that need a certain energy level to reproduce, or that can only do it when they find a mate of a certain age or size. 

Now, detach the current routines of the controller, and attach the ones we just created in addition with the resource spawn and eating mechanisms:

In [54]:
controller.detach_all_routines()
controller.attach_routine(sexual_reproduction)
controller.start_resources_apparition(interval=30)
controller.start_eating_mechanism()

Now, agents that will eat resources will be able to reproduce with a mate, and will do so if they encounter one in their reproduction range. In order to promote the collection of resources to the agents, attach `obstacle_avoidance` and `foraging` behaviors. Then to make them attractive to each other, attach the `love` behavior:

In [55]:
for agent in controller.agents:
    agent.detach_all_behaviors(stop_motors=True)
    agent.attach_behavior(obstacle_avoidance)
    # attach love behavior so agents are attracted to each other
    agent.attach_behavior(love)
    agent.attach_behavior(foraging)

You should now observe a simulation where agents reproduce when they are close to each other and have eaten (with a maximum number of 10 agents).

That's it, don't forget to properly close the session when you have finished:

In [ ]:
controller.stop()
stop_server_and_interface(safe_mode=False)